In [1]:
import pandas as pd
import glob
from funcoes_uteis import *
import numpy as np

# Testando validar_cnpj() e formatar_cnpj()

In [2]:
# Prepara arquivos

# Baixa tabela para testes e a salva
#empresas_listadas_B3 = pd.read_html("https://investidorsardinha.r7.com/geral/todos-os-cnpj-das-empresas-listadas-na-b3-a-bolsa-de-valores-brasileira/",header=0)[0][["Empresa","CNPJ"]]
#empresas_listadas_B3.to_csv("./Inputs/empresas_listadas_B3.csv")

colunas = ["Empresa","CNPJ"]


empresas_listadas_B3 = pd.read_csv("./Inputs/empresas_listadas_B3.csv")[colunas]

# Tira formatação para testar funções. Não é necessário, entretanto, que a coluna seja formatada como inteiro
empresas_listadas_B3.CNPJ = empresas_listadas_B3.CNPJ.str.replace("\W","",regex=True).astype(np.int64)

# Mantém apenas índices ímpares para testar se são retornados os índices corretos
empresas_listadas_B3 = empresas_listadas_B3[empresas_listadas_B3.index % 2 == 1]

# Muda CNPJ para número alétório
empresas_listadas_B3.loc[3,"CNPJ"] = 123456789

empresas_listadas_B3.head()

,Empresa,CNPJ
1,AES TIETE ENERGIA SA,4128563000110
3,ALEF S.A.,123456789
5,ALGAR TELECOM S/A,71208516000174
7,ALLPARK EMPREENDIMENTOS PARTICIPACOES SERVICOS...,60537263000166
9,ALPER CONSULTORIA E CORRETORA DE SEGUROS S.A.,11721921000160


In [3]:
#  Formata CNPJs, checa validade mas não retorna erro
empresas_listadas_B3.CNPJ = formatar_cnpj(empresas_listadas_B3.CNPJ,check_val=True,on_error="pass")

2 CNPJs inválidos:


3      00.000.123/4567-89
285    67.260.377/0001-14
Name: CNPJ, dtype: object

In [4]:
# Com a coluna formatada, pode-se guardar a o resultado da validação
empresas_listadas_B3["Validade"] = validar_cnpj(empresas_listadas_B3.CNPJ)
empresas_listadas_B3.head()

,Empresa,CNPJ,Validade
1,AES TIETE ENERGIA SA,04.128.563/0001-10,True
3,ALEF S.A.,00.000.123/4567-89,False
5,ALGAR TELECOM S/A,71.208.516/0001-74,True
7,ALLPARK EMPREENDIMENTOS PARTICIPACOES SERVICOS...,60.537.263/0001-66,True
9,ALPER CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.721.921/0001-60,True


# Testando formatar_cpf() validar_cpf()

In [13]:
# CPFs aleatórios
cpfs = pd.DataFrame({"CPF":[91488364001,98195998062,36935326052,84010039035,6513862094,12345678910,99999999999,11111111111,pd.NA,""]})

cpfs["CPF_formatado"] = formatar_cpf(cpfs.CPF)
cpfs["Validade"] = validar_cpf(cpfs.CPF_formatado)

cpfs

,CPF,CPF_formatado,Validade
0,91488364001,914.883.640-01,True
1,98195998062,981.959.980-62,True
2,36935326052,369.353.260-52,True
3,84010039035,840.100.390-35,True
4,6513862094,065.138.620-94,True
5,12345678910,123.456.789-10,False
6,99999999999,999.999.999-99,False
7,11111111111,111.111.111-11,False
8,<NA>,000.000.000-00,False
9,,000.000.000-00,False


# Testando sheet_to_parquet()

In [5]:
# Prepara arquivos
empresas_listadas_B3 = pd.read_csv("./Inputs/empresas_listadas_B3.csv")[colunas]
empresas_listadas_B3[empresas_listadas_B3.index %2 == 0].to_csv("./Inputs/impar_par/empresas_listadas_B3_par.csv")
empresas_listadas_B3[empresas_listadas_B3.index %2 == 1].to_csv("./Inputs/impar_par/empresas_listadas_B3_impar.csv")

In [6]:
file_paths = glob.glob("./Inputs/impar_par/*.csv")

In [7]:
# Lê arquivos, concatena, e cria arquivos parquet
empresas_listadas_B3_impar_e_par = sheet_to_parquet(file_paths,colunas,colunas_sem_repetir=["CNPJ"],ignore_parquets=True)

Lendo arquivo: ./Inputs/impar_par\empresas_listadas_B3_impar.csv
./Inputs/impar_par/Parquets/
Salvando em diretório: ./Inputs/impar_par/Parquets/
Criando arquivo: ./Inputs/impar_par/Parquets/empresas_listadas_B3_impar.gzip
Lendo arquivo: ./Inputs/impar_par\empresas_listadas_B3_par.csv
./Inputs/impar_par/Parquets/
Salvando em diretório: ./Inputs/impar_par/Parquets/
Criando arquivo: ./Inputs/impar_par/Parquets/empresas_listadas_B3_par.gzip


In [8]:
# Ao tentar ler novamente, lê-se os parquets
empresas_listadas_B3_impar_e_par = sheet_to_parquet(file_paths,colunas,colunas_sem_repetir=["CNPJ"])
empresas_listadas_B3_impar_e_par.head()

Lendo arquivo: ./Inputs/impar_par/Parquets/empresas_listadas_B3_impar.gzip
Lendo arquivo: ./Inputs/impar_par/Parquets/empresas_listadas_B3_par.gzip


,Empresa,CNPJ
0,AES TIETE ENERGIA SA,04.128.563/0001-10
1,ALEF S.A.,02.217.319/0001-07
2,ALGAR TELECOM S/A,71.208.516/0001-74
3,ALLPARK EMPREENDIMENTOS PARTICIPACOES SERVICOS...,60.537.263/0001-66
4,ALPER CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.721.921/0001-60
